# Federated Learning

## Frecency Sampling

To be able to quickly prototype the federated learning algorithm, a dataset is required.
This notebook is based on a fake frecency dataset that was designed to be very interpretable and at the same time close to the actual data.
The assumption for the data generation is that the current frecency algorithm is perfect. By sampling based on this axiom, we can check if the algorithm actually works.

In [1]:
import numpy as np
from random import random

### Sampling the model input

These are weights that describe how common certain features are. For `recency` we assume a uniform distribution over time, for `type` numbers were chosen that intuitively seem to be reasonable.

In [2]:
type_weights = {
    "visited": 0.6,
    "typed": 0.2,
    "bookmarked": 0.1,
    "other_type": 0.1
}

In [3]:
recency_weights = {
    "4-days": 0.03,
    "14-days": 0.05,
    "31-days": 0.1,
    "90-days": 0.32,
    "other_recency": 0.5
}

For the simulation, it seems to be a fair assumption that `type` and `recency` are independent of each other.
This means we can just multiply the probabilities.

This is probably not completely true, since users likely visit bookmarks more often, but it makes things easier here and the probabilities are hard to estimate well anyways.

In [4]:
def combine_dicts_multiplicatively(dict1, dict2):
    """
    A new dict is created that contains all pairs of the two input dicts.
    The values correspond to the products of the values.
    """
    weights = {}

    for key1, weight1 in dict1.items():
        for key2, weight2 in dict2.items():
            key = (key1, key2)
            weight = weight1 * weight2
            weights[key] = weight
            
    return weights

In [5]:
weights = combine_dicts_multiplicatively(type_weights, recency_weights)

A one-hot representation makes it easier to implement the rest of the formulas. numpy allows us to generate this easily using a permutation of the identity matrix.

In [6]:
def one_hot(num_choices, vector):
    return np.eye(num_choices)[vector]

The input vector to the model is 20-dimensional: One field for every combination of `type` and `recency`.
In the frecency algorithm, we consider the last ten visits to the URL.
Thus, the sum of all elements of the vector is a natural number between 1 and 10.

In [7]:
def sample_weighted(num_samples, weight_dict):
    """Randomly sample from a dict using the values as probabilities"""
    num_choices = len(weight_dict)
    choice_weights = weight_dict.values()
    samples = np.random.choice(num_choices, num_samples, p=choice_weights)
    return one_hot(num_choices, samples)

In [8]:
def sample_url_features(num_samples):
    return sample_weighted(num_samples, weights)

### Sampling the target labels

These are the weights found in the current frecency algorithm. Based on the one-hot encoding, this is just a linear function.

In [9]:
type_points = {
    "visited": 1.2,
    "typed": 2,
    "bookmarked": 1.4,
    "other_type": 0
}

In [10]:
recency_points = {
    "4-days": 100,
    "14-days": 70,
    "31-days": 50,
    "90-days": 30,
    "other_recency": 10
}

In [11]:
frecency_points_dict = combine_dicts_multiplicatively(type_points, recency_points)

In [12]:
# To make sure that the order of keys is the same everywhere
key_order = weights.keys()
frecency_points = np.array([frecency_points_dict[key] for key in key_order])

After all these preparations, an arbitrary number of frecency scores can be computed using a single matrix multiplication.

In [13]:
def frecency(url_features):
    return url_features.dot(frecency_points)

Finally, we are sampling from the above distributions and then call the frecency function.

In [14]:
def sample(num_samples):
    X = sample_url_features(num_samples)
    y = frecency(X)
    return X, y

## Linear Regression

In [15]:
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet

To fit a model, we sample a lot of these scores and also add noise on top to make the problem more similar to the real application.

In [16]:
n = int(1e6)
noise = np.random.normal(0, 2, size=(n))
X, y = sample(n)
y += noise

In [17]:
model = LinearRegression(fit_intercept=False)

In [18]:
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

The resulting coefficients are extremely close to the actual frecency weights. How close they are depends on how much noise we add to the data matrix.

In [19]:
zip(model.coef_, frecency_points)

[(120.00740659137126, 120.0),
 (36.004578642260974, 36.0),
 (0.021075250379209241, 0.0),
 (98.028286000507634, 98.0),
 (14.023348451486875, 14.0),
 (20.014400627354195, 20.0),
 (0.0024597203014381187, 0.0),
 (140.00408148593684, 140.0),
 (-0.078562045852951831, 0.0),
 (70.024480698407487, 70.0),
 (199.99585230509933, 200.0),
 (42.027334912205404, 42.0),
 (99.99929024256339, 100.0),
 (0.010894382049548968, 0.0),
 (59.992257784618502, 60.0),
 (140.03507955810906, 140.0),
 (12.000968581417023, 12.0),
 (59.997861448712243, 60.0),
 (83.993722997329556, 84.0),
 (-0.038365851254008314, 0.0)]

In [20]:
model.coef_ - frecency_points

array([ 0.00740659,  0.00457864,  0.02107525,  0.028286  ,  0.02334845,
        0.01440063,  0.00245972,  0.00408149, -0.07856205,  0.0244807 ,
       -0.00414769,  0.02733491, -0.00070976,  0.01089438, -0.00774222,
        0.03507956,  0.00096858, -0.00213855, -0.006277  , -0.03836585])